In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=fda04244a64ad6e2d570887a7882c9d7f08b761ea7c82607f89ad34ce2881b69
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, Concatenate, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

import pandas_ta as ta
import requests

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

##Get the data from api

In [ ]:
import yfinance as yf
yf.pdr_override()

def fetch_data(symbol):
  df = yf.Ticker(symbol)
  df = df.history(period="max")
  df['AdjClose'] = df['Close']
  del df['Dividends']
  del df['Stock Splits']
  return df

df = fetch_data("BSL")
df

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


,Open,High,Low,Close,Volume,AdjClose
Date,,,,,,
2010-05-26 00:00:00-04:00,7.178959,7.182548,7.178959,7.178959,261400,7.178959
2010-05-27 00:00:00-04:00,7.178959,7.182548,7.178959,7.178959,66700,7.178959
2010-05-28 00:00:00-04:00,7.178964,7.182553,7.178964,7.182553,42700,7.182553
2010-06-01 00:00:00-04:00,7.178959,7.186138,7.178959,7.178959,42700,7.178959
2010-06-02 00:00:00-04:00,7.178962,7.196909,7.178962,7.193320,106500,7.193320
...,...,...,...,...,...,...
2024-06-03 00:00:00-04:00,14.070000,14.130000,13.960000,13.990000,59700,13.990000
2024-06-04 00:00:00-04:00,13.980000,14.050000,13.920000,13.990000,31500,13.990000
2024-06-05 00:00:00-04:00,13.980000,14.030000,13.920000,14.030000,51200,14.030000


##News Sentiment

In [ ]:
# import requests
# import pandas as pd
# from datetime import datetime

# def fetch_and_parse_sentiment_data(ticker, api_key):
#     api_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}"
#     response = requests.get(api_url)
#     data = response.json()

#     sentiment_records = []
#     for article in data['feed']:
#         time_published = datetime.strptime(article['time_published'], '%Y%m%dT%H%M%S').date()
#         for ticker_sentiment in article['ticker_sentiment']:
#             sentiment_records.append({
#                 'date': time_published,
#                 'ticker': ticker_sentiment['ticker'],
#                 'sentiment_score': float(ticker_sentiment['ticker_sentiment_score']),
#                 'relevance_score': float(ticker_sentiment['relevance_score'])
#             })

#     sentiment_df = pd.DataFrame(sentiment_records)
#     return sentiment_df

# api_key = "JFF5XUTNZIEADCG0"
# ticker = "AMZN"
# sentiment_df = fetch_and_parse_sentiment_data(ticker, api_key)
# # print(sentiment_df.head())
# sentiment_df


In [ ]:
# def aggregate_sentiment_data(sentiment_df):
#     aggregated_sentiment_df = sentiment_df.groupby(['date', 'ticker']).agg(
#         avg_sentiment_score=('sentiment_score', 'mean'),
#         avg_relevance_score=('relevance_score', 'mean')
#     ).reset_index()
#     return aggregated_sentiment_df

# aggregated_sentiment_df = aggregate_sentiment_data(sentiment_df)
# print(aggregated_sentiment_df.head())


In [ ]:
# def merge_stock_and_sentiment(stock_data, sentiment_data, ticker):
#     stock_data = stock_data.reset_index()
#     stock_data['Date'] = stock_data['Date'].dt.date
#     sentiment_data = sentiment_data[sentiment_data['ticker'] == ticker]

#     merged_data = pd.merge(stock_data, sentiment_data, left_on='Date', right_on='date', how='left')
#     merged_data = merged_data.drop(columns=['date', 'ticker'])
#     merged_data = merged_data.fillna(0)  # Fill missing sentiment scores with 0
#     merged_data.set_index('Date', inplace=True)
#     return merged_data

# ticker = 'AMZN'  # Example ticker
# merged_data = merge_stock_and_sentiment(df, aggregated_sentiment_df, ticker)
# print(merged_data.tail())


##create technical indicators

In [ ]:
# Function to create technical indicators
def create_technical_indicators(data):
    data['MA10'] = ta.sma(data['Close'], length=10)
    data['MA50'] = ta.sma(data['Close'], length=50)
    data['EMAF'] = ta.ema(data.Close, length=20)
    data['EMAM'] = ta.ema(data.Close, length=100)
    data['EMAS'] = ta.ema(data.Close, length=150)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['MACD'], data['MACD_signal'], _ = ta.macd(data['Close'])
    return data


##preprocess data

In [ ]:
def preprocess_data(data, features, sequence_length, augment_data=True):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features])
    X, y = [], []
    for i in range(len(data) - sequence_length):
        if augment_data:
            start_index = np.random.randint(0, len(data) - sequence_length)
            end_index = start_index + sequence_length
            X.append(scaled_data[start_index:end_index])
            y.append(1 if data['TargetMoving'].values[end_index] > 0 else 0)
        else:
            X.append(scaled_data[i:i+sequence_length])
            y.append(1 if data['TargetMoving'].values[i+sequence_length] > 0 else 0)
    return np.array(X), np.array(y)


##Build The Model

In [ ]:
def build_lstm_model(input_shape, num_features):
    lstm_input = Input(shape=input_shape)
    x = Bidirectional(LSTM(units=64, return_sequences=True))(lstm_input)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(units=32, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = LSTM(units=16)(x)
    output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=lstm_input, outputs=output)
    return model

##Train the model

In [ ]:
def train_stock_model(stock_symbol, sequence_length=30, features=None, augment_data=True):
    # Fetch stock data
    data = fetch_data(stock_symbol)
    data = create_technical_indicators(data)
    data['TargetMoving'] = data['Close'].shift(-1) - data['Close']
    data.dropna(inplace=True)

    # Define features if not specified
    if features is None:
        # features = ['Open', 'High', 'Low', 'Close', 'EMAF', 'EMAM', 'EMAS', 'RSI']
        features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'RSI']


    # Preprocess data
    X, y = preprocess_data(data, features, sequence_length, augment_data=augment_data)
    X, y = shuffle(X, y, random_state=42)  # Shuffle the data to remove any underlying order

    split_ratio = 0.8
    split_index = int(split_ratio * len(X))

    X_train = X[:split_index]
    y_train = y[:split_index]
    X_test = X[split_index:]
    y_test = y[split_index:]

    # Build LSTM model
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]), len(features))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    # Train the model
    model.fit(X_train, y_train,
              validation_data=(X_test, y_test),
              epochs=50,
              batch_size=64,
              callbacks=[early_stopping, reduce_lr],
              verbose=1)

    AUC, y_pred, y_prop = evaluate_model(model, X_test, y_test)
    y_test = y_test.ravel()
    y_pred = y_pred.ravel()
    y_prop = y_prop.ravel()

    # Create DataFrame to compare actual vs predicted
    results = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred,'y_prop': y_prop})
    print(results)
    print("evaluation: ", AUC)
    print(stock_symbol)

    return model


##Train with sentiment

In [ ]:

# # Update train_stock_model to include sentiment in features
# def train_stock_model(stock_symbol, sequence_length=30, features=None, augment_data=True):
#     # Fetch stock data
#     data = fetch_data(stock_symbol)
#     data = create_technical_indicators(data)
#     data['TargetMoving'] = data['Close'].shift(-1) - data['Close']
#     data.dropna(inplace=True)

#     # Fetch sentiment data and merge
#     sentiment_data = fetch_and_parse_sentiment_data(stock_symbol, api_key)
#     processed_sentiment_data = aggregate_sentiment_data(sentiment_data)
#     data = merge_stock_and_sentiment(data, processed_sentiment_data, stock_symbol)

#     # Define features if not specified
#     if features is None:
#         features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA10', 'MA50', 'RSI', 'avg_sentiment_score']

#     # Preprocess data
#     X, y = preprocess_data(data, features, sequence_length, augment_data=augment_data)
#     X, y = shuffle(X, y, random_state=42)  # Shuffle the data to remove any underlying order

#     split_ratio = 0.8
#     split_index = int(split_ratio * len(X))

#     X_train = X[:split_index]
#     y_train = y[:split_index]
#     X_test = X[split_index:]
#     y_test = y[split_index:]

#     # Build LSTM model
#     model = build_lstm_model((X_train.shape[1], X_train.shape[2]), len(features))
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

#     # Callbacks for early stopping and learning rate reduction
#     early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

#     # Train the model
#     model.fit(X_train, y_train,
#               validation_data=(X_test, y_test),
#               epochs=50,
#               batch_size=64,
#               callbacks=[early_stopping, reduce_lr],
#               verbose=1)

#     mse, y_pred = evaluate_model(model, X_test, y_test)
#     y_test = y_test.ravel()
#     y_pred = y_pred.ravel()

#     # Create DataFrame to compare actual vs predicted
#     results = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})
#     print(results)
#     print("evaluation: ", mse)
#     print(stock_symbol)

#     return model

In [ ]:
def train_models_for_bulk_stocks(stock_symbols):
    trained_models = {}
    for stock_symbol in stock_symbols:
        model = train_stock_model(stock_symbol)
        trained_models[stock_symbol] = model
    return trained_models

##evaluate model

In [ ]:
from sklearn.metrics import roc_auc_score
# Function to evaluate model
def evaluate_model(model, X_test, y_test):
    y_prp = model.predict(X_test)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    auc = roc_auc_score(y_test, y_pred)
    print(f"AUC-ROC: {auc}")

    return auc, y_pred, y_prp

##predict

In [ ]:
# Function to predict tomorrow's movement for a stock symbol
def predict_tomorrow_movement(stock_symbol, trained_models, sequence_length=30, features=None):
    # Fetch latest data for the specified stock symbol
    data = fetch_data(stock_symbol)
    latest_data = create_technical_indicators(data)
    model = trained_models[stock_symbol]
    # Prepare input data for the model
    # latest_features = ['Open', 'High', 'Low', 'Close', 'EMAF', 'EMAM', 'EMAS', 'RSI']
    latest_features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'RSI']

    if features is not None:
        latest_features = features

    latest_data = latest_data[latest_features][-sequence_length:]
    # print(latest_data)
    scaler = MinMaxScaler()
    latest_data = scaler.fit_transform(latest_data)
    # print(latest_data)
    latest_data = np.expand_dims(latest_data, axis=0)
    # print(latest_data)

    # Make prediction
    prediction = model.predict(latest_data)
    predicted_movement = "Up" if prediction > 0.5 else "Down"

    return predicted_movement,prediction

In [ ]:
def predict_tomorrow_movement_for_symbols(stock_symbols,trained_models,sequence_length=30, features=None):
    predictions = {}
    for symbol in stock_symbols:
        model = trained_models
        prediction,probabilty = predict_tomorrow_movement(symbol, model, sequence_length, features)
        predictions[symbol] = prediction,probabilty[0][0]
    return predictions

In [ ]:
def train_models_for_bulk_stocks(stock_symbols):
    trained_models = {}
    for stock_symbol in stock_symbols:
        model = train_stock_model(stock_symbol)
        trained_models[stock_symbol] = model
    return trained_models


In [ ]:
# Rank stocks based on predictions
def rank_stocks(bulk_predictions):
    ranked_stocks = sorted(bulk_predictions.items(), key=lambda x: x[1], reverse=True)
    return ranked_stocks

In [ ]:
# Return top 10 stocks as investment opportunities
def get_top_10_investment_opportunities(ranked_stocks):
    return ranked_stocks[:10]

In [ ]:
SAU = [
    "4310.SR", "2222.SR", "2380.SR", "2381.SR", "4030.SR", "4200.SR", "1201.SR", "1202.SR", "1210.SR", "1211.SR",
    "1301.SR", "1304.SR", "1320.SR", "1321.SR", "1322.SR", "2001.SR", "2010.SR", "2020.SR", "2060.SR", "2090.SR",
    "2150.SR", "2170.SR", "2180.SR", "2200.SR", "2210.SR", "2220.SR", "2223.SR", "2240.SR", "2250.SR", "2290.SR",
    "2300.SR", "2310.SR", "2330.SR", "2350.SR", "2360.SR", "3001.SR", "3002.SR", "3003.SR", "3004.SR", "3005.SR",
    "3007.SR", "3008.SR", "3010.SR", "3020.SR", "3030.SR", "3040.SR", "3050.SR", "3060.SR", "3080.SR", "3090.SR",
    "3091.SR", "1212.SR", "1214.SR", "1302.SR", "1303.SR", "2040.SR", "2110.SR", "2160.SR", "2320.SR", "2370.SR",
    "4110.SR", "4140.SR", "4141.SR", "4142.SR", "1831.SR", "1832.SR", "1833.SR", "4270.SR", "6004.SR", "2190.SR",
    "4031.SR", "4040.SR", "4260.SR", "4261.SR", "1213.SR", "2130.SR", "2340.SR", "4011.SR", "4012.SR", "4180.SR",
    "1810.SR", "1820.SR", "1830.SR", "4170.SR", "4290.SR", "4291.SR", "4292.SR", "6002.SR", "6012.SR", "6013.SR",
    "6014.SR", "6015.SR", "4070.SR", "4071.SR", "4210.SR", "4003.SR", "4008.SR", "4050.SR", "4051.SR", "4190.SR",
    "4191.SR", "4192.SR", "4240.SR", "4001.SR", "4006.SR", "4061.SR", "4160.SR", "4161.SR", "4162.SR", "4163.SR",
    "4164.SR", "2050.SR", "2100.SR", "2270.SR", "2280.SR", "2281.SR", "2282.SR", "2283.SR", "4080.SR", "6001.SR",
    "6010.SR", "6020.SR", "6040.SR", "6050.SR", "6060.SR", "6070.SR", "6090.SR", "2140.SR", "2230.SR", "4002.SR",
    "4004.SR", "4005.SR", "4007.SR", "4009.SR", "4013.SR", "4014.SR", "2070.SR", "4015.SR", "1010.SR", "1020.SR",
    "1030.SR", "1050.SR", "1060.SR", "1080.SR", "1120.SR", "1140.SR", "1150.SR", "1180.SR", "1111.SR", "1182.SR",
    "1183.SR", "2120.SR", "4081.SR", "4082.SR", "4130.SR", "4280.SR", "8010.SR", "8012.SR", "8020.SR", "8030.SR",
    "8040.SR", "8050.SR", "8060.SR", "8070.SR", "8100.SR", "8120.SR", "8150.SR", "8160.SR", "8170.SR", "8180.SR",
    "8190.SR", "8200.SR", "8210.SR", "8230.SR", "8240.SR", "8250.SR", "8260.SR", "8270.SR", "8280.SR", "8300.SR",
    "8310.SR", "8311.SR", "7200.SR", "7201.SR", "7202.SR", "7203.SR", "7204.SR", "7010.SR", "7020.SR", "7030.SR",
    "7040.SR", "2080.SR", "2081.SR", "2082.SR", "2083.SR", "5110.SR", "4330.SR", "4331.SR", "4332.SR", "4333.SR",
    "4334.SR", "4335.SR", "4336.SR", "4337.SR", "4338.SR", "4340.SR", "4342.SR", "4344.SR", "4345.SR", "4346.SR",
    "4347.SR", "4348.SR", "4349.SR", "4020.SR", "4090.SR", "4100.SR", "4150.SR", "4220.SR", "4230.SR", "4250.SR",
    "4300.SR", "4320.SR", "4321.SR", "4322.SR",
]


USA = [
    "MSFT", "AAPL", "NVDA", "GOOG", "GOOGL", "AMZN", "META", "LLY", "AVGO", "JPM", "V", "WMT", "XOM", "UNH",
    "MA", "PG", "HD", "ORCL", "COST", "MRK", "CVX", "ABBV", "CRM", "NFLX", "AMD", "KO", "PEP", "GE", "LIN",
    "ADBE", "TMO", "DIS", "MCD", "TMUS", "ABT", "QCOM", "DHR", "AMAT", "INTU", "CMCSA", "UBER", "NOW", "COP",
    "UNP", "NKE", "PM", "MU", "ISRG", "SCHW"
]

UK = [
    "AZN", "LYG", "HSBC", "UL", "BP", "GSK", "STLA", "RELX", "DEO", "BTI", "NGG", "FERG", "HLN",
    "BCS", "CCEP", "NWG", "WTW", "PUK", "VOD", "APTV", "IHG", "CNHI", "NVT", "SNN", "ROIV", "PSO",
    "LBTYB", "LBTYK", "LBTYA", "JHG", "ATCOL", "CLVT", "ARBKL", "NOMD", "LIVN", "TRMD", "IMCR",
    "MANU", "AY", "DAVA", "VTEX", "VRNA", "AUTL", "CNTA", "GSM", "BCYC", "BSIG"
]

AUS = [
    "BHP", "CBA", "RIO", "CSL",
    # "WBC",
    "WES", "WDS", "RMD", "TLS", "WOW", "ALL", "JHX", "COL", "SUN", "AMC",
    "ORG", "IAG", "FPH", "MIN", "CAR", "ASX", "SOL",
    # "TLC",
    "APA",
    # "PME",
    "BSL"
]

JAP = [
    "TM", "MUFG", "8035.T", "SONY", "6861.T",
    # "NTT",
    "8058.T", "6501.T", "4063.T", "9983.T",
    "9984.T", "8316.T", "8031.T", "6098.T",
    "8001.T", "8766.T", "9433.T", "HMC", "4568.T",
    "6902.T", "4519.T", "8411.T", "2914.T", "4502.T", "6367.T", "6503.T", "3382.T", "5108.T",
    "8053.T", "6702.T", "6178.T", "8801.T", "7011.T", "8002.T",
    # "CAJ",
    "6954.T", "8725.T",
    "6857.T", "6301.T", "4901.T", "4543.T", "8802.T", "9022.T", "7269.T", "8015.T", "4578.T",
    "8750.T", "9020.T", "6752.T", "9613.T", "5401.T"
]

# trained_models = train_models_for_bulk_stocks(JAP)
# test = ["AMZN","TSLA"]
test = ["WBC", "BHP", "6010.SR", "9020.T"]

trained_models = train_models_for_bulk_stocks(JAP)

Epoch 1/50
138/138 [==============================] - 31s 134ms/step - loss: 0.6859 - auc: 0.5508 - val_loss: 0.6894 - val_auc: 0.5446 - lr: 0.0010
Epoch 2/50
138/138 [==============================] - 15s 111ms/step - loss: 0.6822 - auc: 0.5570 - val_loss: 0.6891 - val_auc: 0.5167 - lr: 0.0010
Epoch 3/50
138/138 [==============================] - 17s 121ms/step - loss: 0.6819 - auc: 0.5570 - val_loss: 0.6827 - val_auc: 0.5676 - lr: 0.0010
Epoch 4/50
138/138 [==============================] - 15s 108ms/step - loss: 0.6816 - auc: 0.5578 - val_loss: 0.6839 - val_auc: 0.5596 - lr: 0.0010
Epoch 5/50
138/138 [==============================] - 16s 114ms/step - loss: 0.6801 - auc: 0.5621 - val_loss: 0.6833 - val_auc: 0.5742 - lr: 0.0010
Epoch 6/50
138/138 [==============================] - 16s 116ms/step - loss: 0.6802 - auc: 0.5651 - val_loss: 0.6833 - val_auc: 0.5788 - lr: 0.0010
Epoch 7/50
138/138 [==============================] - 15s 111ms/step - loss: 0.6802 - auc: 0.5615 - val_loss: 0.

In [ ]:
stock_symbol = 'AMZN'
prediction,probabilty = predict_tomorrow_movement(stock_symbol, trained_models)
print(prediction)

1/1 [==============================] - 0s 108ms/step
Up


In [ ]:
predictions = predict_tomorrow_movement_for_symbols(USA,trained_models)
predictions_df = pd.DataFrame(predictions)
print(predictions_df)

1/1 [==============================] - 0s 27ms/step
       MSFT      AAPL      NVDA      GOOG     GOOGL      AMZN      META  \
0      Down        Up        Up        Up        Up        Up      Down   
1  0.454057  0.563808  0.556853  0.546047  0.575309  0.546648  0.471518   

        LLY      AVGO       JPM  ...    CMCSA      UBER       NOW       COP  \
0        Up        Up        Up  ...       Up      Down        Up        Up   
1  0.561797  0.554998  0.569082  ...  0.54272  0.499366  0.554582  0.522228   

        UNP       NKE        PM        MU      ISRG      SCHW  
0        Up        Up      Down        Up        Up      Down  
1  0.524106  0.520665  0.497418  0.537333  0.571431  0.494059  

[2 rows x 49 columns]


##Saving  The Models

In [ ]:
for symbol, model in trained_models.items():
    model.save(f'{symbol}_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os

# Define the directory path
directory = r"trained_models"

# Ensure the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Assuming you have a dictionary named 'trained_models' containing your trained models
for symbol, model in trained_models.items():
    filepath = os.path.join(directory, f"{symbol}_model.h5")
    model.save(filepath)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os

# Define the directory path to be downloaded
directory_to_download = "/content/trained_models"

# Define the path where the zip file will be created
zip_file_path = "/content/models.zip"

# Compress the folder into a zip file
shutil.make_archive('/content/models', 'zip', directory_to_download)

# Move the zip file to the specified path
shutil.move("/content/models.zip", zip_file_path)

# Download the zip file
from google.colab import files
files.download(zip_file_path)
